In [1]:
# matlotlib inline plotting
%matplotlib inline

In [10]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np

from utils import maybe_download as maybe_download


In [11]:
url = 'http://ufldl.stanford.edu/housenumbers/'

In [ ]:
maybe_download(url, 'train.tar.gz', 404141560)
maybe_download(url, 'test.tar.gz', 276555967)
maybe_download('extra.tar.gz', 1955489752)

maybe_download(url, 'train_32x32.mat', 0)
maybe_download(url, 'test_32x32.mat', 0)
maybe_download(url, 'extra_32x32.mat', 0)

Attempting to download: train.tar.gz
0%....5%....10%....15%....20%....25%....30%....35%....40%....45%....50%....55%....60%....65%....70%....75%....80%....85%....90%....95%....100%
Download Complete!
Found and verified train.tar.gz
Attempting to download: test.tar.gz
0%....5%....10%....15%....20%....25%....30%....35%....40%....45%....50%....55%....60%....65%....70%....75%....80%....85%....90%....95%....100%
Download Complete!
Found and verified test.tar.gz
Attempting to download: train_32x32.mat
0%....5%....10%....15%....20%....25%....30%....35%....40%....45%....50%....55%....60%....65%....70%....75%....80%....85%....90%....95%....100%
Download Complete!
Failed to verify train_32x32.mat. Please delete and try again
Attempting to download: test_32x32.mat
0%....5%....10%....15%....20%....25%....30%....35%....40%....45%....50%....55%....60%....65%....70%....75%....80%....85%....90%....95%....100%
Download Complete!
Failed to verify test_32x32.mat. Please delete and try again
Attempting to 